In [5]:
import os, traceback, requests, urllib3
import pandas as pd
from dotenv import load_dotenv # pip install python-dotenv
from slack_sdk import WebClient # pip install slack-sdk

assert load_dotenv('config.ini') == True


In [2]:
class Crawler:
    
    def __init__(self, CONN_RETRY_COUNT=4, CONN_STATUS_FORCELIST=[429, 500, 502, 503, 504]):
        
        self.CONN_RETRY_COUNT = CONN_RETRY_COUNT
        self.CONN_STATUS_FORCELIST = CONN_STATUS_FORCELIST
        
    
    def __conn__(self):
        
        session = requests.Session()
        adapter = requests.adapters.HTTPAdapter(max_retries=urllib3.Retry(total=self.CONN_RETRY_COUNT, backoff_factor=1, allowed_methods=None, status_forcelist=self.CONN_STATUS_FORCELIST))
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        
        return session
    
    def __send__(self, method, url, verify=False, timeout=5, params={}, headers={}, **kwargs):
        
        session = self.__conn__()
        
        if method == 'GET':
            response = session.get(url=url, headers=headers, verify=verify, timeout=timeout, params=params)
        elif method == 'POST':
            data = kwargs['data']
            response = session.post(url=url, headers=headers, verify=verify, timeout=timeout, data=data)

        if not 'response' in locals():
            return -999, 'ERROR'
        
        return response.status_code, response


In [3]:
class SlackInfo:
    
    TOKEN_SLACK = os.environ['TOKEN_SLACK']
    
    def __init__(self):
        self.client = self.__conn__()
        
    def __conn__(self):
        return WebClient(
            token=self.TOKEN_SLACK, 
            # ssl=sslcert, e.g. from ssl import SSLContext; sslcert = SSLContext()
            # proxy=proxyinfo # e.g. "http://localhost:9000"
        )
            
    def get_channels_list(self):
        return self.client.conversations_list()['channels']
    
    def get_users_list(self):
        return self.client.users_list()['members']


class Notify:

    TOKEN_LINE_NOTIFY = os.environ['TOKEN_LINE_NOTIFY']
    TOKEN_SLACK = os.environ['TOKEN_SLACK']

    URL_LINE_NOTIFY = os.environ['URL_LINE_NOTIFY']

    ENABLE_NOTIFY_LINE = eval(os.environ['ENABLE_NOTIFY_LINE'])
    ENABLE_NOTIFY_SLACK = eval(os.environ['ENABLE_NOTIFY_SLACK'])

    # 確保通知相關參數有被宣告
    def __init__(self):
        
        assert self.ENABLE_NOTIFY_LINE != None
        assert self.ENABLE_NOTIFY_SLACK != None

    # 作為發送訊息主控台 - 呼叫各通訊軟體
    def __message__(self, title:str, msg:str, **kwargs):
        if self.ENABLE_NOTIFY_LINE == True:
            self.__line__(msg='{} - {}'.format(title, msg))
        
        if self.ENABLE_NOTIFY_SLACK == True and 'channel' in kwargs:
            tags, channel = '', kwargs['channel']
            
            msg = "*{}*\n{}".format(title, msg)
            if 'members' in kwargs:
                if len(kwargs['members']) >= 1:
                    tags = str(['<@{}>'.format(user_id) for user_id in kwargs['members']]).replace("['", '').replace("']", '')
                    msg += '\n{}'.format(tags)
            
            self.__slack__(msg=msg, channel=channel, mrkdwn=True, is_file=False)

        if self.ENABLE_NOTIFY_LINE == False and self.ENABLE_NOTIFY_SLACK == False:
            print(
                'Display Message - {}\n\t{}'.format(title, msg)
            )

    # 作為發送檔案主控台 - 呼叫 Slack 傳送檔案
    def __fileUpload__(self, title:str, msg:str, channel:str, filename:str):

        if os.path.isfile(filename):
            msg = "*{}*\n{}".format(title, msg)
            self.__slack__(msg=msg, channel=channel, mrkdwn=True, is_file=True, filename=filename)
        else:
            print('[Notify][Slack] - File Not Exist')

    # 發送訊息至 Line Notify
    def __line__(self, msg:str):
        assert self.TOKEN_LINE_NOTIFY != None

        headers = {
            "Authorization": "Bearer " + self.TOKEN_LINE_NOTIFY, 
            "Content-Type" : "application/x-www-form-urlencoded"
        }

        data = {'message': msg}

        status_code, response = Crawler().__send__(method='POST', url=self.URL_LINE_NOTIFY, verify=True, headers=headers, data=data)

        if status_code == 200:
            print('[Notify][Line Notify] - Success')
        else:
            print('[Notify][Line Notify] - Fail')

    # 發送 訊息/檔案 至 Slack
    def __slack__(self, msg:str, channel:str, is_file:bool, mrkdwn:bool, filename=None):
        assert self.TOKEN_SLACK != None
        
        client = WebClient(token=self.TOKEN_SLACK)

        if is_file == False:
            response = client.chat_postMessage(channel=channel, text=msg, mrkdwn=mrkdwn)
        elif is_file == True and os.path.isfile(filename):
            response = client.files_upload_v2(
                channel=channel,
                file=filename,
                initial_comment=msg
            )

        if 'response' in locals():
            status_code = response.get('ok')

            if status_code == True:
                print('[Notify][Slack] - Success')
            else:
                print('[Notify][Slack] - Fail')
                
        else:
            print('[Notify][Slack] - Send Fail')


In [ ]:
class Handler:
    def __init__(self, type_name:str, source_name:str, channel:str, member:str):
        self.type_name = type_name
        self.source_name = source_name
        self.channel = channel
        self.member = member
        
    def __call__(self, func):
        def wrapper(*args, **kwargs):

            try:
                func(*args, **kwargs)
                Notify().__controller__(title='[{}][{}]'.format(self.type_name, self.source_name), msg='Finish', channel=self.channel, members=self.member)

            except Exception as e:
                Notify().__controller__(title='[{}][{}]'.format(self.type_name, self.source_name), msg=traceback.format_exc(), channel=self.channel, members=self.member)

        return wrapper

In [4]:
# Slack - Get Channels and Members
channels = pd.DataFrame(SlackInfo().get_channels_list())
members = pd.DataFrame(SlackInfo().get_users_list())
 
# Example 1. Notify Status
Notify().__controller__(title='[Example 1 - Notify Status]', msg='<@ron.jl.lee>', channel=os.environ['SLACK_CHANNEL'])

# Example 2. Notify Exception
try:
    c
except Exception as e:
    Notify().__controller__(title='[Example 2 - Notify Exception]', msg=traceback.format_exc(), channel=os.environ['SLACK_CHANNEL'])

# Example 3. Upload File to Slack
Notify().__fileUpload__(title='[Example 3 - Upload File to Slack]', msg='Upload file', channel=os.environ['SLACK_CHANNEL'], filename='README.md')

# Example 4. Combine Decorator 
@Handler(type_name='INFO', source_name='Example 4 - Combine Decorator', channel=os.environ['SLACK_CHANNEL'], member=[])
def f(a, b):
    print('calling f with args ', (a, b))
f(a=1, b=2)

# Example 5. Combine Decorator
@Handler(type_name='INFO', source_name='Example 5 - Combine Decorator', channel=os.environ['SLACK_CHANNEL'], member=[])
def f(a, b):
    print('calling f with args ', (a, b, c))
f(a=1, b=2)

[Notify][Line Notify] - Success
[Notify][Slack] - Success
[Notify][Line Notify] - Success
[Notify][Slack] - Success
[Notify][Slack] - Success
calling f with args  (1, 2)
[Notify][Line Notify] - Success
[Notify][Slack] - Success
[Notify][Line Notify] - Success
[Notify][Slack] - Success
